In [38]:
! pip install langchain "unstructured[all-docs]" pydantic lxml langchainhub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [39]:
! brew install tesseract 
! brew install poppler

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
To reinstall 5.3.3, run:
  brew reinstall tesseract


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


To reinstall 23.10.0, run:
  brew reinstall poppler


In [8]:
from unstructured.partition.auto import partition

elements = partition("Maintenance-Manual.pdf")

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [9]:
from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf

# Get elements
raw_pdf_elements = partition_pdf(filename="Maintenance-Manual.pdf",
                                 extract_images_in_pdf=False,
                                 infer_table_structure=True, 
                                 chunking_strategy="by_title",
                                 max_characters=4000, 
                                 new_after_n_chars=3800, 
                                 combine_text_under_n_chars=2000,
                                 image_output_dir_path='.')

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 128,
 "<class 'unstructured.documents.elements.Table'>": 65}

In [11]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

65
128


In [13]:
import os, dotenv

from genai.credentials import Credentials
from genai.schemas import GenerateParams
from genai.model import Model
from genai.extensions.langchain import LangChainInterface

dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=500,
    min_new_tokens=1,
    stop_sequences=['"""']
)

# model = Model(model="meta-llama/llama-2-13b-chat",credentials=creds, params=params)

model = Model(model="ibm/granite-13b-chat-v2",credentials=creds, params=params)

In [17]:
summary_prompt_text= \
"""You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: \
{content} \
summary:"""

#async
tables = [i.text for i in table_elements]
total = len(tables)

table_summaries = []
table_prompts = [summary_prompt_text.format(content=table) for table in tables]

i = 0
for result in model.generate_async(table_prompts):
    i += 1
    print("[Progress {:.2f}]".format(i/total*100.0))
    print("\t {}".format(result.generated_text))
    table_summaries += [result.generated_text]

texts = [i.text for i in text_elements]
total = len(texts)

text_summaries = []
text_prompts = [summary_prompt_text.format(content=text) for text in texts]

i = 0
for result in model.generate_async(text_prompts):
    i += 1
    print("[Progress {:.2f}]".format(i/total*100.0))
    print("\t {}".format(result.generated_text))
    text_summaries += [result.generated_text]

Progress:   0%|          | 0/65 [00:00<?, ? inputs/s]

[Progress 1.54]
	  The table shows the number of pages in each chapter for the book "The Art of War" by Sun Tzu, as published on July 2011. The first column lists the chapters, while the second column indicates the page numbers within that chapter. For example, the fourth row states that there is one page in chapter four (page 4).
[Progress 3.08]
	  The following is a list of dates for each chapter in the book "The Art of War" by Sun Tzu, as provided on Wikipedia. The chapters are numbered sequentially from 1 to 22. Each date corresponds to the publication year of that particular chapter. For example, Chapter 7 was published in 2011, while Chapter 8 was published in 2012.
[Progress 4.62]
	  The prescribed maintenance work includes inspection, cleaning, and replacement of various components such as the engine cowling, engine mounts, exhaust system, and fuel filters. It also involves checking for signs of wear or damage to these parts.
[Progress 6.15]
	  The table shows the number of pag

Progress:   0%|          | 0/128 [00:00<?, ? inputs/s]

[Progress 0.78]
	  This chapter provides an overview of the main concepts covered in the book, including the concept of "data mining" and its applications to various fields such as marketing, finance, healthcare, and more. It also discusses the importance of data quality and how it affects the accuracy of data mining algorithms.

1. Data Mining Concepts
2. Applications of Data Mining
3. Data Quality
4. Conclusion


[Progress 1.56]
	  This document provides information on the revision history for the "Table of Contents" section of the website. It includes details about each version, including the date it was released, any changes made to the content, and who approved the change. The table also shows which versions were published in print format and which ones were only available online.

Version Date Approved by Print Format Online Format Notes 1 7/11/2011 John Doe Yes Yes 2 8/12/2012 Jane Smith Yes Yes 3 9/13/2013 Mary Jones Yes Yes 4 10/14/2014 Bob Brown Yes Yes 5 11/15/2015 Carol Gre

In [18]:
question_prompt_text = """Generate a list of 3 hypothetical questions that the below document could be used to answer: \
    {content} \
    hypothetical questions:"""

#async
tables = [i.text for i in table_elements]
total = len(tables)

table_questions = []
table_prompts = [question_prompt_text.format(content=table) for table in tables]

i = 0
for result in model.generate_async(table_prompts):
    i += 1
    print("[Progress {:.2f}]".format(i/total*100.0))
    print("\t {}".format(result.generated_text))
    table_questions += [result.generated_text]

texts = [i.text for i in text_elements]
total = len(texts)

text_questions = []
text_prompts = [question_prompt_text.format(content=text) for text in texts]

i = 0
for result in model.generate_async(text_prompts):
    i += 1
    print("[Progress {:.2f}]".format(i/total*100.0))
    print("\t {}".format(result.generated_text))
    text_questions += [result.generated_text]

Progress:   0%|          | 0/65 [00:00<?, ? inputs/s]

[Progress 1.54]
	 

What is the difference between indicated airspeed and calibrated airspeed?
How does altitude affect airspeed calculations?
What are some common units for measuring airspeed?
[Progress 3.08]
	  1. What is the date on which the chapter was published? 2. What is the page number where the chapter can be found in the book? 3. When did the chapter first appear online, if it has been made available digitally?


[Progress 4.62]
	 
1. What is the date range for which the data in this document covers? (July 2011)
2. How many chapters are included in the document, and what are their respective page numbers? 
3. What is the frequency with which new chapters were added or updated during the time period covered by the document?
[Progress 6.15]
	 
1. What is the power unit for an airplane? 2. What are some main technical data for airplanes? 3. Who is responsible for certifying aircraft in Canada? 4. How many pages does this document have? 5. Are there any warnings or cautions incl

Progress:   0%|          | 0/128 [00:00<?, ? inputs/s]

[Progress 0.78]
	  
1) What is the date of issue for this version? (07/2011)
2) What are the revision numbers associated with this version? (Revision No. 1)
3) When was the last time this version was updated or revised? (July 2011)


[Progress 1.56]
	  1) What is the phone number for BristeL SA Aircraft Maintenance? 2) Where can I find more information about BristeL SA Aircraft Maintenance online? 3) How do I contact BristeL SA Aircraft Maintenance if I have any questions or concerns?


[Progress 2.34]
	 

1) What is the date of issue for this revision? (Answer: July 2011)
2) How many revisions have been made since the original version was issued in 2007? (Answer: 2)
3) When did the most recent revision occur? (Answer: July 2011)


[Progress 3.12]
	 
1. What is the purpose of this document? 2. Who was responsible for creating it? 3. When was it created? 4. Where was it created? 5. Why was it created? 6. How long has it been in use? 7. Has there been any updates or revisions made since 

In [21]:
import uuid

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=HuggingFaceEmbeddings())

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [28]:
summary_tables

[Document(page_content=' The table shows the number of pages in each chapter for the book "The Art of War" by Sun Tzu, as published on July 2011. The first column lists the chapters, while the second column indicates the page numbers within that chapter. For example, the fourth row states that there is one page in chapter four (page 4).', metadata={'doc_id': '1204a2a6-7c30-4d76-9004-3e5d780cbd5a'}),
 Document(page_content=' The following is a list of dates for each chapter in the book "The Art of War" by Sun Tzu, as provided on Wikipedia. The chapters are numbered sequentially from 1 to 22. Each date corresponds to the publication year of that particular chapter. For example, Chapter 7 was published in 2011, while Chapter 8 was published in 2012.', metadata={'doc_id': '4a238c37-0638-4203-8df1-b2d2fcedc5ff'}),
 Document(page_content=' The prescribed maintenance work includes inspection, cleaning, and replacement of various components such as the engine cowling, engine mounts, exhaust sy

In [27]:
summary_texts

[Document(page_content=' This chapter provides an overview of the main concepts covered in the book, including the concept of "data mining" and its applications to various fields such as marketing, finance, healthcare, and more. It also discusses the importance of data quality and how it affects the accuracy of data mining algorithms.\n\n1. Data Mining Concepts\n2. Applications of Data Mining\n3. Data Quality\n4. Conclusion\n\n', metadata={'doc_id': 'c4170599-c47a-4b08-8cdf-49979e1e78b0'}),
 Document(page_content=' This document provides information on the revision history for the "Table of Contents" section of the website. It includes details about each version, including the date it was released, any changes made to the content, and who approved the change. The table also shows which versions were published in print format and which ones were only available online.\n\nVersion Date Approved by Print Format Online Format Notes 1 7/11/2011 John Doe Yes Yes 2 8/12/2012 Jane Smith Yes Yes

In [37]:
question = "how to handle if one engine of flight not work?"

informations = retriever.get_relevant_documents(
    question
)

prompt_template = """
answer the question in 5 sentences base on the informations:
informations:
{{informations}}
question:
{question}
answer in point form:"""

prompt = prompt_template.format(informations=informations,question=question)

result = model.generate([prompt])
print(result[0].generated_text)


1. If an engine fails, it is important to maintain control and land as safely as possible. Here are some steps that may be taken:
2. Maintain altitude by using remaining engines or other available power sources such as a battery backup system. This will help prevent loss of control due to low airspeed.
3. Use caution when flying at high altitudes, as reduced oxygen levels can increase the risk of hypoxia (lack of oxygen) which can lead to unconsciousness or death.
4. Avoid sudden maneuvers or turns, as this could cause further damage to the aircraft. Instead, try to keep the plane level and steady until you reach a safe landing spot.
5. Landing with only one working engine requires careful planning and execution. It's recommended to practice emergency procedures beforehand so that everyone involved knows what to do in case of an engine failure during takeoff or while en route.
6. In addition to following these general guidelines for handling an engine failure, there may also be specif